# Examples

These are some examples on how to read and write TSDF data into and from a numpy array.

## How to run these examples

### Dependencies

In order to run these examples, we'll need to use the following Python packages:


In [13]:
import tsdf
import os
import numpy as np
from tsdf.constants import TestConstants as TEST_CONST

### Test  files

The test files used in these examples can be found in [`./tests/data/`](https://github.com/biomarkersParkinson/tsdf/tree/main/tests/data). Use the snippet below to locate your copy of the files:

In [14]:
# Reload modules automatically on changes; useful for developing
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Required import

## Process an existing binary file and write the new TSDF metadata
Read and process an existing binary data (accompanied by the TSDF metadata), process the data and save it in the new format, with the corresponding TSDF metadata file.

### Load dummy data and see its format

In [15]:
dummy_data_name = "example_10_3_int16"

dummy_metadata = tsdf.load_metadata_from_path(os.path.join(TEST_CONST.TEST_DATA_DIR, dummy_data_name + TEST_CONST.METADATA_EXTENSION))[dummy_data_name + TEST_CONST.BINARY_EXTENSION]
dummy_data = dummy_metadata.load_binary()
print(f"Data type used for storing:\t {dummy_data.dtype}")
print(f"Data dimensions:\t\t {dummy_data.shape}")
print(f"Number of rows:\t\t\t {dummy_data.shape[0]}")

Data type used for storing:	 int16
Data dimensions:		 (10, 3)
Number of rows:			 10


### Perform light data processing

In [16]:
processed_dummy_data_1 = (dummy_data / 10).astype('float32')
print(f"Data type used for storing:\t {processed_dummy_data_1.dtype}")
print(f"Data dimensions:\t\t {processed_dummy_data_1.shape}")
print(f"Number of rows:\t\t\t {processed_dummy_data_1.shape[0]}")

Data type used for storing:	 float32
Data dimensions:		 (10, 3)
Number of rows:			 10


### Metadata available from the data (NumPy array)
The metadata will be used indirectly to generate the new TSDF metadata file.

In [17]:
bin_meta = tsdf.get_metadata_from_ndarray(processed_dummy_data_1)
print(f"Binary formatting that can be inferred from the NumPy array:\n{bin_meta}")

Binary formatting that can be inferred from the NumPy array:
{'data_type': 'float', 'bits': 32, 'endianness': 'little', 'rows': 10}


### Write the processed data 
Write the processed data in binary format. The call returns the corresponding metadata object.

In [18]:
processed_dummy_data_name_1 = "tmp_test_example_10_3_int16_to_float32"
processed_dummy_metadata_1 = tsdf.write_binary_file(
            TEST_CONST.TEST_OUTPUT_DATA_DIR,
            processed_dummy_data_name_1 + TEST_CONST.BINARY_EXTENSION,
            processed_dummy_data_1,
            dummy_metadata.get_plain_tsdf_dict_copy(),
        )

### Write the TSDF metadata file that describes the processed binary data format

#### 1) Write the metadata file for a single binary file

In [19]:
# Write new metadata file
tsdf.write_metadata([processed_dummy_metadata_1], processed_dummy_data_name_1 + TEST_CONST.METADATA_EXTENSION)


#### 2) Write a metadata file that combines multiple binary files

In [20]:
# Preprocess the original data to generate another data source
processed_dummy_data_2 = (dummy_data * 1000).astype('int32')

# Adjust the metadata slightly
updated_dummy_metadata = dummy_metadata.get_plain_tsdf_dict_copy()
updated_dummy_metadata.pop("scale_factors") #remove the 'scale_factors'


# Save the new binary file
processed_dummy_data_name_2 = "tmp_test_example_10_3_int16_to_int32"
processed_dummy_metadata_2 = tsdf.write_binary_file(
            TEST_CONST.TEST_OUTPUT_DATA_DIR,
            processed_dummy_data_name_2 + TEST_CONST.BINARY_EXTENSION,
            processed_dummy_data_2,
            updated_dummy_metadata,
        )

# Write a metadata file that combines the two binary files
tsdf.write_metadata([processed_dummy_metadata_1, processed_dummy_metadata_2], "tmp_test_example_10_3_int16_to_int_n_float_meta.json")


## Genarate a new binary file and the corresponding TSDF metadata
Generate binary data and save it and the corresponding TSDF metadata file.

In [21]:
rs = np.random.RandomState(seed=42)
data_1 = rs.rand(17, 1).astype(np.float32)
data_2 = rs.rand(15, 2).astype(np.int16)
data_3 = rs.rand(10, 3).astype(np.int16)


# An example where the metadata is defined from scratch
new_metadata = {}
new_metadata["subject_id"] = "example"
new_metadata["study_id"] = "example"
new_metadata["device_id"] = "example"
new_metadata["endianness"] = "little"
new_metadata["metadata_version"] = "0.1"
new_metadata["start_datetime_unix_ms"] = 1571135957025,
new_metadata["start_iso8601"] = "2019-10-15T10:39:17.025000+00:00"
new_metadata["end_datetime_unix_ms"] = 1571168851826
new_metadata["end_iso8601"] = "2019-10-15T19:47:31.826000+00:00"
new_metadata["channels"] = ["x","y","z"]
new_metadata["units"] = ["m/s/s","m/s/s","m/s/s"]

# Write the three binary files based on the provided metadata

file_prefix = "tmp_test"
new_meta_1 =  tsdf.write_binary_file(TEST_CONST.TEST_OUTPUT_DATA_DIR, file_prefix + "_1.bin", data_1, new_metadata)
new_meta_2 =  tsdf.write_binary_file(TEST_CONST.TEST_OUTPUT_DATA_DIR, file_prefix+"_2.bin", data_2, new_metadata)
new_meta_3 =  tsdf.write_binary_file(TEST_CONST.TEST_OUTPUT_DATA_DIR, file_prefix+"_3.bin", data_3, new_metadata)


Save the metadata that corresponds to the binary data. In case of multiple binary files, the corresponding metadata files have to be combined.

In [22]:
# Write the first metadata file
tsdf.write_metadata([new_meta_1], file_prefix + "_1" + TEST_CONST.METADATA_EXTENSION)

In [23]:
# Combine and write all metadata files
tsdf.write_metadata([new_meta_1, new_meta_2, new_meta_3], file_prefix + "_3" + TEST_CONST.METADATA_EXTENSION)

## Transform legacy (TSDB) format to the current TSDF v0.1
Transform one file (or all files within the given directory) from TSDB to TSDF format.

In [24]:
from tsdf.utils_legacy_TSDF import generate_tsdf_metadata_from_tsdb, convert_metadata_tsdb_to_tsdf, convert_metadatas_tsdb_to_tsdf

# Path to the metadata file
path_to_file = os.path.join(TEST_CONST.TEST_DATA_DIR, "ppp_format_meta_legacy.json")
path_to_new_file = os.path.join(TEST_CONST.TEST_OUTPUT_DATA_DIR, "tmp_ppp_format_meta.json")

# Generate a TSDF metadata file from TSDB
generate_tsdf_metadata_from_tsdb(path_to_file, path_to_new_file)

# Convert a TSDB metadata file to TSDB format 
# convert_metadata_tsdb_to_tsdf(path_to_file)

# Convert all metadata files in the directory from TSDB to TSDF format
# convert_metadatas_tsdb_to_tsdf(path_to_dir)